In [ ]:
import pandas as pd
import os
import sys
import pickle
import datetime
import time
import re
from sqlalchemy import create_engine
from sqlalchemy import text
import sqlalchemy
import joblib
from collections import Counter
import numpy as np

In [ ]:
TOKENIZERS_PARALLELISM=False
TF_ENABLE_ONEDNN_OPTS=0

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
STOP_WORDS = { "the",
    "and", "or", "the", "a", "an", "to", "of", "in", "on", "for", "with",
    "by", "at", "from", "as", "is", "are", "this", "that", 'tv', 'collection',
    'episodes' ,'episode', 'и', 'ну', 'не', 'ссср','shorts', 'y', 'kidsmania',
    'un', 'el', 'de', 'be'
}

In [ ]:
def tokenize_clean(text):
    tokens = re.findall(r"\b[^\W_]+\b", text.lower(), flags=re.UNICODE)
    return [ t for t in tokens if not t.isdigit() and t not in STOP_WORDS ]

In [ ]:
df = pd.read_excel('markuped.xlsx')
ids = df.yt_reel_id.to_list()
ids_str = "'" + "','".join(ids) + "'"

In [ ]:
conn_string = os.environ['PG_DB2']
db = create_engine(conn_string.replace('multdb', 'youtube'))
conn = db.connect()
req1 = '''select iyr.yt_reel_id, reel_name, yt_ch_name, cartoon, brand_id, product_id,language from intl_yt_reels iyr 
join intl_yt_channels2reels iyrcr 
on iyr.yt_reel_id = iyrcr.yt_reel_id 
join intl_yt_channels iyc 
on iyrcr.yt_ch_id = iyc.yt_ch_id 
where iyr.yt_reel_id in ({}) '''.format(ids_str)
df_sql = pd.read_sql(text(req1), conn)
conn.close()

In [ ]:
df_sql['name'] = df_sql.yt_ch_name + ' '+df_sql.reel_name
df_sql["tokens"] = df_sql["name"].apply(tokenize_clean)
texts2 = df_sql['tokens'].str.join(' ').to_list()
labels2 = df_sql['brand_id'].to_list()

In [ ]:
pipeline = joblib.load("pipeline.pkl")
y_pred = pipeline.predict(texts2)

In [ ]:
print(classification_report(labels2, y_pred))
print(confusion_matrix(labels2, y_pred))

In [ ]:
final = pd.concat([df_sql,pd.DataFrame(y_pred)], axis =1 )

In [ ]:
final.to_excel('markuped+multiclass_new.xlsx', index = False)